In [64]:
### Q1 hash tree 
### A1_yulaipo_20741014
### input of it 
data="{1 2 3}, {1 4 5}, {1 2 4}, {1 2 5}, {1 5 9}, {1 3 6},\
             {2 3 4}, {2 5 9}, \
             {3 4 5}, {3 5 6}, {3 5 9}, {3 8 9}, {3 2 6}, \
             {4 5 7}, {4 1 8}, {4 7 8}, {4 6 7}, \
             {6 1 3}, {6 3 4}, {6 8 9}, {6 2 1}, {6 4 3}, {6 7 9},\
             {8 2 4}, {8 9 1}, {8 3 6}, {8 3 7}, {8 4 7}, {8 5 1}, {8 3 1},{8 6 2}"
data=list(map(lambda x:list(map(int,x.strip()[1:-1].split())),data.split(",")))

In [65]:
### output the answer
hash_tree(data)

[[[[1, 4, 5], [4, 1, 8], [4, 7, 8]],
  [[[1, 2, 4], [4, 5, 7]], [1, 2, 5], [[1, 2, 3], [1, 5, 9]]],
  [[1, 3, 6], [4, 6, 7]]],
 [[8, 4, 7],
  [[2, 5, 9], [8, 2, 4], [8, 5, 1]],
  [[[2, 3, 4], [8, 9, 1], [8, 3, 7], [8, 3, 1]], [8, 6, 2], [8, 3, 6]]],
 [[[], [3, 4, 5], [[6, 1, 3], [6, 4, 3], [6, 7, 9]]],
  [[6, 2, 1], [], [[3, 5, 6], [3, 5, 9], [3, 8, 9], [3, 2, 6], [6, 8, 9]]],
  [6, 3, 4]]]

In [57]:
### define the max leaf size is 3 as questions required
def hash_tree(T,max_leaf_size=3,i=0):
    if len(T)<=max_leaf_size or i>=len(T[0]):
        return T
    reduce = lambda T:T[0] if len(T)==1 else T
    T1=reduce([x for x in T if x[i]%3==1])
    T2=reduce([x for x in T if x[i]%3==2])
    T3=reduce([x for x in T if x[i]%3==0])
    return [hash_tree(T1,max_leaf_size,i+1),hash_tree(T2,max_leaf_size,i+1),hash_tree(T3,max_leaf_size,i+1)]

In [139]:
### Q2 FP tree
import pandas as pd
import numpy as np
import csv

class define_Node:
    def __init__(SELF, item, degree, parent):
        SELF.item = item
        SELF.degree = degree
        SELF.parent = parent
        SELF.link_node = None
        SELF.children = {}


class FP_Tree:
  
    def __init__(SELF, data_set):
 
        SELF.product_set = data_set

    def FP_tree(SELF, k):
        FP_tree_root = FP_Node('null set', 1, None)
        tables = SELF.check_freq(k)
        
        for transfer, degree in k.items():
            temporary = {}
            for item in transfer:
                if item in tables:
                    temporary[item] = tables[item][0]
            if len(temporary) > 0:
                orderedItems = sorted(temporary, key=temporary.get, reverse=True)
                SELF.update_fp_tree(orderedItems, tables, FP_tree_root,  degree)
        return FP_tree_root, tables


    def check_freq(SELF, product_set):
        tables = {}
        for transfer in product_set:
            number = product_set[transfer]
            while number:
                for item in transfer:
                    tables[item] = tables.get(item, 0) + 1
                number -= 1
        for key in set(tables.keys()):
            if tables[key] < 2500:
                del tables[key]

        for key in tables:
            value = tables[key]
            tables[key] = [value, None]
        return tables


    def update_fp_tree(SELF, new_dataset, tables, FP_tree_root, degree):
        if new_dataset[0] in FP_tree_root.children:
            FP_tree_root.children[new_dataset[0]].degree += degree
        else:
            FP_tree_root.children[new_dataset[0]] = FP_Node(new_dataset[0], degree, FP_tree_root)
            if tables[new_dataset[0]][1] == None:
                tables[new_dataset[0]][1] = FP_tree_root.children[new_dataset[0]]
            else:
                node = tables[new_dataset[0]][1]
                while(node.link_node != None):
                    node = node.link_node
                node.link_node = FP_tree_root.children[new_dataset[0]]
        if len(new_dataset) > 1:
            SELF.update_fp_tree(new_dataset[1:], tables, FP_tree_root.children[new_dataset[0]], degree)

        return tables


    def cross(SELF, FP_Node):
        
        output = {}
        while FP_Node != None:
            temporary_node = FP_Node
            path = []
            while temporary_node.parent != None:
                path.append(temporary_node.item)
                temporary_node = temporary_node.parent
            if len(path) > 1:
                output[frozenset(path[1:])] = FP_Node.degree
            FP_Node = FP_Node.link_node
        return output


    def remove(SELF, FP_tree_root, tables, prefix, freq_itemset, FP_tree_root_set):
        for item in tables.keys():
            temporary = prefix.copy()
            temporary.add(item)
            freq_itemset.append(temporary)
            new_dataset = SELF.cross(tables[item][1])
            new_FP_tree_root, new_head = SELF.FP_tree(new_dataset)
            FP_tree_root_set.append(new_FP_tree_root)
            if new_head != None:
                SELF.remove(new_FP_tree_root, new_head, temporary, freq_itemset, FP_tree_root_set)


    def find_fp_tree(SELF, node, tree):
        if FP_Node != None:
            tree.append([node.item+'    '+str(node.degree)])
            if node.children != None:
                for child in node.children:
                    SELF.find_fp_tree(node.children[child], tree)

if __name__ == '__main__':
    with open('DataSetA.csv') as a:
        data = list(csv.reader(a))
    product_set = {}
    for row in data[:]:
        while '' in row:
            row.remove('')
        row = frozenset(i for i in row)
        product_set[row] = product_set.get(row, 0) + 1
    tree = FP_Tree(product_set)
    FP_tree_root, tables = tree.FP_tree(product_set)
    freq_itemset = []
    prefix = set([])
    FP_tree_root_set = []
    tree.remove(FP_tree_root, tables, prefix, freq_itemset, FP_tree_root_set)
    with open("DataSetA,csv", "w") as a:
        writer = csv.writer(a)
        for item in freq_itemset:
            writer.writerow([item])
  
    for FP_tree_root in FP_tree_root_set:
        output = []
        tree.find_fp_tree(FP_tree_root, output)
        if len(output) > 2:
            print(output)


[['null set    1'], ['Ghee    2578'], ['Milk    1197'], ['Milk    1321']]
[['null set    1'], ['Yougurt    2532'], ['Coffee Powder    630'], ['Bread    1203'], ['Coffee Powder    577'], ['Bread    1327'], ['Coffee Powder    636'], ['Coffee Powder    674']]
[['null set    1'], ['Coffee Powder    2555'], ['Milk    1237'], ['Milk    1276']]
[['null set    1'], ['Coffee Powder    659'], ['Ghee    329'], ['Butter    58'], ['Bread    154'], ['Sugar    73'], ['Butter    38'], ['Butter    44'], ['Sugar    73'], ['Butter    33'], ['Bread    148'], ['Butter    42'], ['Sugar    69'], ['Butter    30'], ['Sugar    71'], ['Butter    25'], ['Butter    47'], ['Butter    64'], ['Milk    1309'], ['Coffee Powder    622'], ['Ghee    318'], ['Sugar    77'], ['Butter    34'], ['Bread    150'], ['Sugar    81'], ['Butter    42'], ['Butter    30'], ['Butter    48'], ['Bread    151'], ['Sugar    70'], ['Butter    34'], ['Butter    42'], ['Butter    24'], ['Sugar    82'], ['Butter    43'], ['Butter    57'], ['Gh